In [8]:
!pip install pandasql

You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.


In [8]:
!pip install ipython-sql

You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.


In [5]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from pandasql import sqldf
from sqlalchemy import create_engine

from datetime import datetime
from datetime import timedelta

In [7]:
##################
# import raw data
##################
df = pd.read_csv('https://uwmadison.box.com/shared/static/6ff20fwvm1ejtk4wfo37e9o7e60y7y5d.gz')
df

,start_time,installation_id,station_id
0,2021-04-01 00:00:00.088007,412914,1025503
1,2021-04-01 00:00:00.107122,413884,985795
2,2021-04-01 00:00:00.199838,418743,876082
3,2021-04-01 00:00:00.244472,419447,883379
4,2021-04-01 00:00:00.352282,413584,855284
...,...,...,...
2066168,2021-04-01 23:59:59.494044,413406,972273
2066169,2021-04-01 23:59:59.513663,412357,969521
2066170,2021-04-01 23:59:59.602626,415752,1005312
2066171,2021-04-01 23:59:59.625039,420719,957148


In [8]:
######################
# create db connection
######################
connection_to_db = create_engine('sqlite:///associations.db')


In [9]:
# table from raw data
df.to_sql('associations', con=connection_to_db, if_exists='replace')

In [10]:
# run first time
%load_ext sql
%sql sqlite:///associations.db

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


In [6]:
# otherwise run this
%reload_ext sql
%sql sqlite:///associations.db

In [7]:
# confirm associations table
%%sql
select * from associations limit 5;

SyntaxError: invalid syntax (<ipython-input-7-fcc88468e33e>, line 3)

In [ ]:
# run once time
# builds associations_cross table
# format:
# station_id, ap_1, ap_2, t1, t2
%%sql

create table associations_cross as select
a.station_id, 
a.installation_id as ap_1,
b.installation_id as ap_2,
a.start_time as t1, 
b.start_time as t2
from associations a join associations b on a.station_id = b.station_id;

In [7]:
# run once time
%%sql
delete from associations_cross where ap_1 = ap_2

 * sqlite:///associations.db
1342052527 rows affected.


[]

In [34]:
%%sql
pragma table_info(associations_cross);

 * sqlite:///associations.db
Done.


cid,name,type,notnull,dflt_value,pk
0,station_id,INT,0,None,0
1,ap_1,INT,0,None,0
2,ap_2,INT,0,None,0
3,t1,TEXT,0,None,0
4,t2,TEXT,0,None,0


In [ ]:
# function to compute time given two timestamps
def time_apart(t1, t2):
  str1 = t1.split()
  str2 = t2.split()
  FMT = '%H:%M:%S.%f'
  tdelta = datetime.strptime(str2[1], FMT) - datetime.strptime(str1[1], FMT)
  return tdelta

In [ ]:
# pandas dataframe
transitions = %sql select * from associations_cross
transitions_df = transitions.DataFrame()

 * sqlite:///associations.db
Done.


In [ ]:
transitions_df

In [ ]:
transitions_df['distance']

In [35]:
# add column for distance apart


SyntaxError: invalid syntax (<ipython-input-35-3d7e1ed60d82>, line 3)

In [ ]:
# compute from t1 and t2


In [ ]:
# drop time stamps and station ids


In [ ]:
# join rows where ap_1 = ap_1 and ap_2 = ap_2
# sum distances
# add column for count


In [8]:
%%sql
select * from associations_cross limit 5

 * sqlite:///associations.db
Done.


station_id,ap_1,ap_2,t1,t2
1025503,412914,413772,2021-04-01 00:00:00.088007,2021-04-01 00:00:21.692255
1025503,412914,412996,2021-04-01 00:00:00.088007,2021-04-01 00:00:29.930613
1025503,412914,416512,2021-04-01 00:00:00.088007,2021-04-01 00:00:34.193936
1025503,412914,415462,2021-04-01 00:00:00.088007,2021-04-01 00:00:48.611063
1025503,412914,415462,2021-04-01 00:00:00.088007,2021-04-01 00:01:08.961677


In [22]:
# get unique installation ids
installations = %sql select installation_id from associations group by installation_id
installations_df = installations.DataFrame()
labels = installations_df['installation_id'].tolist()

 * sqlite:///associations.db
Done.


In [19]:
# init dissimilairty matrix
dissimilarity_matrix = pd.DataFrame(index=labels, columns=labels)
dissimilarity_matrix = dissimilarity_matrix.fillna(0)
dissimilarity_matrix

,1106,411997,411998,411999,412000,412001,412002,412003,412005,412006,...,424250,424251,424252,424253,424254,424255,424256,424257,424258,424259
1106,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
411997,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
411998,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
411999,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
412000,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
424255,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
424256,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
424257,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
424258,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [21]:
dissimilarity_matrix[411998][1106]

0

In [ ]:
# build dissimilarity matrix
# df_associations_cross has cols: station_id, ap_1, ap2, t1, t2
for i, row in data.iterrows():
  